In [1]:
import os
import shared
import networkx as nx
import numpy as np

import pyximport; pyximport.install()

import fennel


try:
    import config
except ImportError as err:
    print(err)
    print("**Could not load config.py\n**Copy config_template.py and rename it.")

pwd = os.getcwd()

DATA_FILENAME = os.path.join(pwd, "data", "oneshot_fennel_weights.txt")
OUTPUT_DIRECTORY = os.path.join(pwd, "output")

# Read input file for prediction model, if not provided a prediction
# model is made using FENNEL
PREDICTION_MODEL = ""

# File containing simulated arrivals. This is used in simulating nodes
# arriving at the shelter. Nodes represented by line number; value of
# 1 represents a node as arrived; value of 0 represents the node as not
# arrived or needing a shelter.
SIMULATED_ARRIVAL_FILE = os.path.join(pwd, "data", "simulated_arrival.txt")
#SIMULATED_ARRIVAL_FILE = ""

# File containing the geographic location of each node.
POPULATION_LOCATION_FILE = os.path.join(pwd, "data", "population_location.csv")

# Number of shelters
num_partitions = 4

# The number of iterations when making prediction model
num_iterations = 10

# Percentage of prediction model to use before discarding
# When set to 0, prediction model is discarded, useful for one-shot
prediction_model_cut_off = 0.10

# Alpha value used in one-shot (when restream_batches set to 1)
one_shot_alpha = 0.5

# Number of arrivals to batch before recalculating alpha and restreaming.
# When set to 1, one-shot is used with alpha value from above
restream_batches = 10

# Create virtual nodes based on prediction model
use_virtual_nodes = False

# Virtual nodes: edge weight
virtual_edge_weight = 1.0


####
# GRAPH MODIFICATION FUNCTIONS

# Also enables the edge calculation function.
graph_modification_functions = True

# If set, the node weight is set to 100 if the node arrives at the shelter,
# otherwise the node is removed from the graph.
alter_arrived_node_weight_to_100 = True

# Uses generalized additive models from R to generate prediction of nodes not
# arrived. This sets the node weight on unarrived nodes the the prediction
# given by a GAM.
# Needs POPULATION_LOCATION_FILE to be set.
alter_node_weight_to_gam_prediction = True

gam_k_value = 100

# Alter the edge weight for nodes that haven't arrived. This is a way to
# de-emphasise the prediction model for the unknown nodes.
prediction_model_emphasis = 1.0


SCOTCH Environment valid.
SCOTCH python bindings were loaded correctly from ../csap-graphpartitioning/src/python 
SCOTCH Library was located successfully at ../csap-graphpartitioning/tools/scotch/lib/macOS/libscotch.dylib


In [2]:
# read METIS file
G = shared.read_metis(DATA_FILENAME)

# Alpha value used in prediction model
prediction_model_alpha = G.number_of_edges() * (num_partitions / G.number_of_nodes()**2)

# Order of nodes arriving
arrival_order = list(range(0, G.number_of_nodes()))

# Arrival order should not be shuffled if using GAM to alter node weights
#random.shuffle(arrival_order)

if SIMULATED_ARRIVAL_FILE == "":
    # mark all nodes as needing a shelter
    simulated_arrival_list = [1]*G.number_of_nodes()
else:
    with open(SIMULATED_ARRIVAL_FILE, "r") as ar:
        simulated_arrival_list = [int(line.rstrip('\n')) for line in ar]
        
print("Graph loaded...")
print("Nodes: {}".format(G.number_of_nodes()))
print("Edges: {}".format(G.number_of_edges()))
if nx.is_directed(G):
    print("Graph is directed")
else:
    print("Graph is undirected")

Graph loaded...
Nodes: 1000
Edges: 2939
Graph is undirected


In [3]:
# setup for other algorithms
if config.ENABLE_SCOTCH == True:
    # import the relevant SCOTCH modules
    from scotch.graph_mapper import GraphMapper
    from scotch.io import ScotchGraphArrays

UNMAPPED = -1

# reset
assignments = np.repeat(np.int32(UNMAPPED), G.number_of_nodes())
fixed = np.repeat(np.int32(UNMAPPED), G.number_of_nodes())

print("PREDICTION MODEL")
print("----------------")

# Begin computation of prediction model
if PREDICTION_MODEL:
    # if we have a prediction model from file, load it
    with open(PREDICTION_MODEL, "r") as inf:
        assignments = np.fromiter(inf.readlines(), dtype=np.int32)

else:
    # SCOTCH algorithm
    # we have a networkx graph already, G
    scotchArrays = ScotchGraphArrays() # create the object storing all the SCOTCH arrays
    scotchArrays.fromNetworkxGraph(G, baseval=0) # populate arrays from G

    #scotchArrays.debugPrint() # uncomment this to print out contents of scotchArrays

    # create instance of SCOTCH Library
    mapper = GraphMapper(config.SCOTCH_LIB_PATH)

    # set some optional parameters for the SCOTCH_Arch, SCOTCH_Strat, SCOTCH_Graph
    # see csap-graphpartitioning/src/python/scotch/graph_mapper: GraphMapper.__init__() method for more options
    mapper.kbalval = 0.1
    mapper.numPartitions = num_partitions

    # intializes the SCOTCH_Arch, SCOTCH_Strat, SCOTCH_Graph using scotchArray and optional parameters
    ok = mapper.initialize(scotchArrays, verbose=False)
    if(ok):
        # we can proceed with graphMap, the data structures were setup correctly
        ok = mapper.graphMap()
        if(ok):
            # graphMap was run successfully, copy the assignments
            # make a deep copy as we then delete the mapper data, to clear memory
            # and the array reference may be lost
            assignments = np.array(mapper.scotchData._parttab, copy=True)

            mapper.delObjects()
        else:
            print('Error while running graphMap()')
    else:
        print('Error while setting up SCOTCH for partitioning.')

x = shared.score(G, assignments, num_partitions)
edges_cut, steps = shared.base_metrics(G, assignments)
print("WASTE\t\tCUT RATIO\tEDGES CUT\tCOMM VOLUME")
print("{0:.5f}\t\t{1:.10f}\t{2}\t\t{3}".format(x[0], x[1], edges_cut, steps))

print("\nAssignments:")
shared.fixed_width_print(assignments)

nodes_fixed = len([o for o in fixed if o == 1])
print("\nFixed: {}".format(nodes_fixed))

shared.print_partitions(G, assignments, num_partitions)

# save the prediction model
initial_model = np.array(assignments, copy=True)

PREDICTION MODEL
----------------
WASTE		CUT RATIO	EDGES CUT	COMM VOLUME
0.05200		0.0214358625	63		107

Assignments:
[ 3  1  2  0  0  0  0  2  3  1  3  1  0  1  1  0  3  2  0  0  1  2  3  2  1  3  2  3  0  1  0  2  0  3  3  0  1  3  2  1  2  1  3  1  1  2  1  3  3  2  0  3  3  2  3  2  0  0  1  0  2  0  1  2  1  1  3  2  1  3  1  0  0  1  3  2  0  3  3  0  2  3  0  2  1  3  1  0  1  0  0  3  3  0  0  2  3  2  1  3  1  0  2  0  1  1  1  0  1  3  2  0  2  3  0  1  3  1  0  2  1  1  0  0  0  3  0  2  0  1  0  3  1  3  3  1  1  2  3  3  1  2  2  0  2  1  3  2  0  3  0  1  1  0  1  2  1  2  3  0  1  1  0  0  1  3  1  3  3  1  3  1  2  3  2  1  1  0  0  1  2  0  1  2  3  1  2  1  0  2  3  2  3  0  0  1  3  0  1  0  2  3  3  1  0  2  2  1  0  2  3  3  3  0  0  0  2  0  2  2  1  1  1  1  2  2  2  0  0  0  1  2  3  3  3  1  3  0  0  1  2  2  0  3  3  2  2  1  0  1  1  3  0  3  3  3  3  2  3  0  3  3  0  2  3  1  0  2  2  3  3  3  1  3  1  2  0  1  2  1  2  3  3  2  1  1  0  2  0  0  0  1  0  1 

In [4]:
if use_virtual_nodes or True:
    print("Creating virtual nodes and assigning edges based on prediction model")

    # create virtual nodes
    virtual_nodes = list(range(G.number_of_nodes(), G.number_of_nodes() + num_partitions))
    print("\nVirtual nodes:")

    # create virtual edges
    virtual_edges = []
    for n in range(0, G.number_of_nodes()):
        virtual_edges += [(n, virtual_nodes[assignments[n]])]

    # extend assignments
    assignments = np.append(assignments, np.array(list(range(0, num_partitions)), dtype=np.int32))
    fixed = np.append(fixed, np.array([1] * num_partitions, dtype=np.int32))

    G.add_nodes_from(virtual_nodes, weight=1)
    G.add_edges_from(virtual_edges, weight=virtual_edge_weight)

    print("\nAssignments:")
    shared.fixed_width_print(assignments)
    print("Last {} nodes are virtual nodes.".format(num_partitions))


Creating virtual nodes and assigning edges based on prediction model

Virtual nodes:

Assignments:
[ 3  1  2  0  0  0  0  2  3  1  3  1  0  1  1  0  3  2  0  0  1  2  3  2  1  3  2  3  0  1  0  2  0  3  3  0  1  3  2  1  2  1  3  1  1  2  1  3  3  2  0  3  3  2  3  2  0  0  1  0  2  0  1  2  1  1  3  2  1  3  1  0  0  1  3  2  0  3  3  0  2  3  0  2  1  3  1  0  1  0  0  3  3  0  0  2  3  2  1  3  1  0  2  0  1  1  1  0  1  3  2  0  2  3  0  1  3  1  0  2  1  1  0  0  0  3  0  2  0  1  0  3  1  3  3  1  1  2  3  3  1  2  2  0  2  1  3  2  0  3  0  1  1  0  1  2  1  2  3  0  1  1  0  0  1  3  1  3  3  1  3  1  2  3  2  1  1  0  0  1  2  0  1  2  3  1  2  1  0  2  3  2  3  0  0  1  3  0  1  0  2  3  3  1  0  2  2  1  0  2  3  3  3  0  0  0  2  0  2  2  1  1  1  1  2  2  2  0  0  0  1  2  3  3  3  1  3  0  0  1  2  2  0  3  3  2  2  1  0  1  1  3  0  3  3  3  3  2  3  0  3  3  0  2  3  1  0  2  2  3  3  3  1  3  1  2  0  1  2  1  2  3  3  2  1  1  0  2  0  0  0  1  0  1  2  3  2  0  3  1 

In [5]:
cut_off_value = int(prediction_model_cut_off * G.number_of_nodes())
if prediction_model_cut_off == 0:
    print("Discarding prediction model\n")
else:
    print("Assign first {} arrivals using prediction model, then discard\n".format(cut_off_value))

# fix arrivals
nodes_arrived = []
for a in arrival_order:
    # check if node needs a shelter
    if simulated_arrival_list[a] == 0:
        continue

    # set 100% node weight for those that need a shelter
    if alter_arrived_node_weight_to_100:
        G.node[a]['weight'] = 100

    nodes_fixed = len([o for o in fixed if o == 1])
    if nodes_fixed >= cut_off_value:
        break
    fixed[a] = 1
    nodes_arrived.append(a)

# remove nodes not fixed, ie. discard prediction model
for i in range(0, len(assignments)):
    if fixed[i] == -1:
        assignments[i] = -1

x = shared.score(G, assignments, num_partitions)
edges_cut, steps = shared.base_metrics(G, assignments)
print("WASTE\t\tCUT RATIO\tEDGES CUT\tCOMM VOLUME")
print("{0:.5f}\t\t{1:.10f}\t{2}\t\t{3}".format(x[0], x[1], edges_cut, steps))

print("\nAssignments:")
shared.fixed_width_print(assignments)

nodes_fixed = len([o for o in fixed if o == 1])
print("\nFixed: {}".format(nodes_fixed))

shared.print_partitions(G, assignments, num_partitions)

Assign first 100 arrivals using prediction model, then discard

WASTE		CUT RATIO	EDGES CUT	COMM VOLUME
7.17131		0.3531353135	1391		1014

Assignments:
[-1  1  2  0 -1  0  0 -1 -1 -1  3 -1  0  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1 -1 -1  0 -1 -1  2 -1  3 -1 -1  1 -1 -1  1 -1  1 -1  1 -1  2  1 -1  3  2  0 -1 -1 -1 -1  2  0 -1  1  0 -1 -1 -1 -1  1 -1  3  2  1  3  1 -1 -1 -1 -1 -1  0 -1 -1  0 -1 -1 -1 -1  1 -1  1 -1 -1  0  0  3 -1  0  0 -1 -1 -1  1 -1  1 -1 -1 -1 -1  1 -1  0 -1  3 -1  0 -1 -1 -1  1  3  1 -1 -1  1 -1  0  0 -1  3 -1 -1  0  1  0 -1  1 -1 -1 -1  1 -1  3  3  1  2  2 -1 -1  1 -1  2 -1 -1 -1 -1  1  0  1  2  1 -1 -1 -1  1  1  0 -1  1  3  1  3 -1 -1 -1  1 -1  3 -1  1 -1 -1 -1  1 -1  0 -1 -1  3  1 -1  1 -1 -1 -1 -1 -1 -1  0  1 -1 -1  1  0 -1 -1 -1  1  0 -1 -1  1 -1 -1 -1 -1  3  0 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 

In [6]:
if restream_batches == 1:
    print("One-shot assignment mode")
    print("------------------------\n")
else:
    print("Assigning in batches of {}".format(restream_batches))
    print("--------------------------------\n")

    
# preserve original node/edge weight
if graph_modification_functions:
    node_weights = {n[0]: n[1]['weight'] for n in G.nodes_iter(data=True)}
    nx.set_node_attributes(G, 'weight_orig', node_weights)

    edge_weights = {(e[0], e[1]): e[2]['weight'] for e in G.edges_iter(data=True)}
    nx.set_edge_attributes(G, 'weight_orig', edge_weights)

Assigning in batches of 10
--------------------------------



In [7]:
G2 = nx.Graph()
for node in G.nodes():
    G2.add_node(node)

for node in G.nodes():
    for neighbor in G.neighbors(node):
        G2.add_edge(node, neighbor)

print(G2.number_of_nodes())
print(G2.number_of_edges())

scotch_assignments = [] # list of all the current assignments
scotch_arrived_nodes = [] # list of the nodes that have arrived
scotch_removed = []


import utilities.alg_utils as algutils

restream_batches = 10

pick_from_prediction = True # if both of these false, picks a random one
pick_from_minfill = False # overrides pick_from_prediction
use_fixed_nodes = True

finalG = None

current_batch = []
batchCount = 1
for i, a in enumerate(arrival_order):
    
    if fixed[a] == 1 and use_fixed_nodes:
        scotch_arrived_nodes.append(a)
        scotch_assignments.append(assignments[a])
        #print('fixed', a, assignments[a])
        continue
    
    # TODO skip invalid nodes
    if graph_modification_functions:
        if simulated_arrival_list[a] == 0:
            # node does not arrive
            #G2.remove_node(a) # deletes all the edges related to this node
            scotch_removed.append(a)
            continue

        if alter_arrived_node_weight_to_100:
            #print('node_weight_altered')
            G.node[a]['weight'] = 100
    
    current_batch.append(a)
    
    # check if enough people have arrived to fill the batch or if this is the last batch
    if restream_batches == len(current_batch) or i == len(arrival_order) - 1:
        print('*******\nBatch #' + str(batchCount), 'arrived. Total nodes =', str(len(current_batch) + len(scotch_arrived_nodes)))
        '''
            Steps
            1. add new nodes and empty assignments from the batch
            2. add all the nodes in scotch_arrived_nodes to this empty graph
            3. add all the edges in scotch_arrived_nodes to the graph
        
        '''
        # step 1
        for newN in current_batch:
            scotch_arrived_nodes.append(newN)
            scotch_assignments.append(-1)
        
        # step 2
        G3 = nx.Graph()
        for node in scotch_arrived_nodes:
            G3.add_node(node)
        
        #print('G3.nodes', G3.nodes())
        
        # step 3: create all the edges that we can already create
        for node in G3.nodes():
            for neighbor in G.neighbors(node):
                # check neighbor is linked to a node in G3
                if neighbor != node and neighbor in scotch_arrived_nodes:
                    # add edge
                    G3.add_edge(node, neighbor)
                    
        #print('G3.baseedge', G3.edges())
        
        # determine which nodes are edgeless - create virtual edges
        virtualEdges = {}
        unassignedVirtualNodes = []
        
        nodes = G3.nodes()
        
        for i, node in enumerate(nodes):
            if scotch_assignments[i] >= 0:
                # node is fixed, no need to create a virtual edge?
                # TODO test this
                continue
            
            if len(G3.neighbors(node)) == 0:
                # edgeless node
                # pick a partition
                partition = algutils.pickRandPartition(num_partitions)
                if pick_from_prediction:
                    partition = initial_model[node]

                if pick_from_minfill:
                    partition = algutils.minPartitionCounts(scotch_assignments, num_partitions)
                
                
                # pick another node in the same partition
                otherNode = algutils.getOtherNodeInPartition(node, partition, scotch_arrived_nodes, scotch_assignments)
                
                # default: connect to virtual nodes for minfill & rand
                
                if use_virtual_nodes:
                    otherNode = G.number_of_nodes() - 4 + partition
                    #print('othernode', otherNode)
                
                if otherNode is None:
                    # place in waiting list
                    unassignedVirtualNodes.append(node)
                else:
                    # create edge between these two nodes
                    uv = algutils.uvOrder(node, otherNode)
                    # store this edge
                    #algutils.addEdgeToIndex(virtualEdges, uv)
                    #print('Adding virt edge', uv)
                    if uv[0] in virtualEdges:
                        virtualEdges[uv[0]].append(uv[1])
                    else:
                        virtualEdges[uv[0]] = [uv[1]]
                    # add edge to graph
                    G3.add_edge(uv[0], uv[1])
                if node in current_batch:
                    #print('Edgeless unassigned', node)
                    # assign to the partition
                    scotch_assignments[algutils.getIndex(scotch_arrived_nodes, node)] = partition

        #print('assignments_updated', scotch_assignments)
        #print('virtual_edges', virtualEdges)
        #print('unaissingedNodes', unassignedVirtualNodes)
        #print('G3.nodes_updated',len(G3.nodes()), G3.nodes())
        #print('G3.edges_updated',len(G3.edges()), G3.edges())
        
        # checks
        for n in unassignedVirtualNodes:
            found = False
            for edge in G3.edges():
                if edge[0] == n or edge[1] == n:
                    found = True
            if(found == False):
                print("WARNING: unassigned Edge found?", n, found)
        
        
        
        
        # update node mapping for SCOTCH
        G4 = algutils.updateNodeMapping(G3, G)
        #print('G4',len(G4.nodes()), G4.nodes())
        #print('G4',len(G4.edges()), G4.edges())
        
        # update node weights and edge weights from G
        
        
        
        print(scotch_assignments)

        # partition
        scotchArrayData = ScotchGraphArrays()
        scotchArrayData.fromNetworkxGraph(G4, parttab=scotch_assignments, baseval=0)

        #print(scotchArrayData.velotab)

        scotchMapper = GraphMapper(config.SCOTCH_LIB_PATH, numPartitions=num_partitions)
        ok = scotchMapper.initialize(scotchArrayData, skipGraphValidStep=False, verbose=False)
        if(ok):
            # mapper initialized
            ok = scotchMapper.graphMapFixed()
            if(ok):
                scotch_assignments = scotchMapper.scotchData._parttab.tolist()
                print(scotch_assignments)
            else:
                print("Error running graphMapFixed()")
        else:
            print("Error initializing SCOTCH GraphMapper for graphMapFixed()")
        
        #print('assignments_after', scotch_assignments)
        
        finalG = G4
        # remove all virtual edges
        print('Removing', len(virtualEdges), 'virtual edges.')
        algutils.delVirtualEdges(finalG, virtualEdges)
        
        
        current_batch = []
        batchCount += 1
        #if len(scotch_arrived_nodes) == 3 * restream_batches:
        #    break
s_assignments = np.array(scotch_assignments)
x = shared.score(finalG, s_assignments, num_partitions)
edges_cut, steps = shared.base_metrics(finalG, s_assignments)
print("WASTE\t\tCUT RATIO\tEDGES CUT\tCOMM VOLUME")
print("{0:.5f}\t\t{1:.10f}\t{2}\t\t{3}".format(x[0], x[1], edges_cut, steps))        

print("\nAssignments:")
shared.fixed_width_print(s_assignments)

shared.print_partitions(finalG, s_assignments, num_partitions)

1004
3939
*******
Batch #1 arrived. Total nodes = 106
[1, 2, 0, 0, 0, 3, 0, 1, 1, 1, 0, 2, 3, 1, 1, 1, 1, 2, 1, 3, 2, 0, 2, 0, 1, 0, 1, 3, 2, 1, 3, 1, 0, 0, 1, 1, 0, 0, 3, 0, 0, 1, 1, 1, 0, 3, 0, 1, 3, 1, 1, 0, 0, 3, 0, 1, 0, 1, 1, 3, 3, 1, 2, 2, 1, 2, 1, 0, 1, 2, 1, 1, 1, 0, 1, 3, 1, 3, 1, 3, 1, 1, 0, 3, 1, 1, 0, 1, 1, 0, 1, 0, 1, 3, 0, 0, -1, -1, 1, 1, -1, 0, 1, -1, 3, 1]
[1, 2, 0, 0, 0, 3, 0, 1, 1, 1, 0, 2, 3, 1, 1, 1, 1, 2, 1, 3, 2, 0, 2, 0, 1, 0, 1, 3, 2, 1, 3, 1, 0, 0, 1, 1, 0, 0, 3, 0, 0, 1, 1, 1, 0, 3, 0, 1, 3, 1, 1, 0, 0, 3, 0, 1, 0, 1, 1, 3, 3, 1, 2, 2, 1, 2, 1, 0, 1, 2, 1, 1, 1, 0, 1, 3, 1, 3, 1, 3, 1, 1, 0, 3, 1, 1, 0, 1, 1, 0, 1, 0, 1, 3, 0, 0, 2, 2, 1, 1, 2, 0, 1, 2, 3, 1]
Removing 6 virtual edges.
*******
Batch #2 arrived. Total nodes = 116
[1, 2, 0, 0, 0, 3, 0, 1, 1, 1, 0, 2, 3, 1, 1, 1, 1, 2, 1, 3, 2, 0, 2, 0, 1, 0, 1, 3, 2, 1, 3, 1, 0, 0, 1, 1, 0, 0, 3, 0, 0, 1, 1, 1, 0, 3, 0, 1, 3, 1, 1, 0, 0, 3, 0, 1, 0, 1, 1, 3, 3, 1, 2, 2, 1, 2, 1, 0, 1, 2, 1, 1, 1, 0, 1, 3, 1, 3,

In [8]:
verttab = [0, 3, 6]
edgetab = [1,2,3, 7,8,9, ...]